You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [6]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:
Title: Exploring the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying up-to-date with the latest trends and key players in the AI market

Key Points:
1. Latest Trends in Artificial Intelligence
- Advancements in deep learning algorithms
- Increased adoption of AI in healthcare, finance, and transportation industries
- Emphasis on ethical AI development and

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a transformative force in various industries, revolutionizing processes and enhancing efficiency. From healthcare to finance and transportation, the impact of AI is undeniable. Staying informed about the latest trends and key players in the AI market is crucial for professionals and enthusiasts alike to navigate the rapidly evolving landscape of technology.

## Latest Trends in Artificial Intelligence
The advancements in deep learning algorithms have propelled AI to new heights, enabling machines to learn and adapt without explicit programming. This has led to increased adoption of AI in key sectors such as healthcare, where it assists in diagnostics and personalized treatment plans. Moreover, there is a growing emphasis on ethical AI development and regulation to ensure responsible and transparent use of AI technologies.

## Key Players in the AI Market
Leading the charge in AI research and development are tech giants like Google, Amazon, Microsoft, and IBM. These companies have invested heavily in AI to drive innovation and create cutting-edge solutions. In addition to established players, startups such as OpenAI and UiPath are making significant contributions to the AI industry with their disruptive technologies. Collaboration between academia and industry further fuels innovation in AI, paving the way for groundbreaking advancements.

## Noteworthy News in Artificial Intelligence
Recent developments in natural language processing and computer vision have pushed the boundaries of AI capabilities, enabling machines to understand and interpret human language and visual data. AI applications in cybersecurity and fraud detection have enhanced threat detection and prevention measures, safeguarding sensitive information. However, the impact of AI on job automation and workforce transformation raises concerns about the future of work and the need for reskilling and upskilling initiatives.

## Conclusion
In conclusion, staying informed about the latest trends and key players in Artificial Intelligence is essential for navigating the dynamic landscape of technology. By understanding the advancements in AI, ethical considerations, and industry applications, professionals and enthusiasts can harness the power of AI for positive change. I encourage readers to explore further resources on AI advancements and engage in discussions about the ethical implications of AI development.

## Call to Action
For more insights on Artificial Intelligence trends, key players, and ethical considerations, I recommend exploring resources such as the "The State of Artificial Intelligence in 2021" report by Forbes and the "Ethical AI Principles" guidelines by the IEEE. Stay informed and be part of the conversation shaping the future of AI.

This blog post aims to provide valuable insights into the latest trends, key players, and noteworthy news in Artificial Intelligence, catering to business professionals, technology enthusiasts, and students interested in AI. By incorporating relevant SEO keywords and reputable resources, the post offers a comprehensive overview of the dynamic AI landscape.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "Space Exploration"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Space Exploration.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Space Exploration

1. Introduction
- Brief overview of the history of space exploration
- Importance of space exploration in scientific advancements and technological developments

2. Latest Trends and Key Players
- SpaceX: notable for its advancements in reusable rocket technology
- NASA Artemis Program: aiming to return humans to the moon by 2024
- Commercial space tourism: companies like Blue Origin and Virgin Galactic offering space travel experiences

3. Noteworthy News
- Recent Mars rover missio

In [18]:
Markdown(result)

# Exploring the Final Frontier: A Journey Through Space Exploration

## Introduction

Space exploration has always captivated the imagination of humanity, pushing the boundaries of what is possible and expanding our understanding of the universe. From the early days of the space race to the groundbreaking advancements of today, the quest to explore the cosmos has been a driving force in scientific advancements and technological developments. The importance of space exploration cannot be overstated, as it has led to breakthroughs in various fields and continues to inspire future generations to reach for the stars.

## Latest Trends and Key Players

In recent years, notable players like SpaceX have revolutionized the space industry with their innovative approach to reusable rocket technology. Companies like Blue Origin and Virgin Galactic are also making strides in commercial space tourism, offering the possibility of experiencing space travel firsthand. On the governmental side, NASA's Artemis Program is aiming to return humans to the moon by 2024, while international collaborations like the International Space Station (ISS) showcase the power of working together in space exploration endeavors.

## Noteworthy News

Recent Mars rover missions have provided us with valuable insights into the red planet, while advancements in satellite technology have enhanced our ability to observe Earth and communicate globally. International collaborations in space exploration continue to push the boundaries of what is possible, paving the way for future missions to the moon, Mars, and beyond. The future of space travel and colonization holds endless possibilities, with exciting developments on the horizon that will shape the way we explore the final frontier.

## Conclusion

Looking towards the future of space exploration, it is evident that the possibilities are endless. From the latest trends and key players in the space industry to the groundbreaking advancements in technology and science, the journey through space exploration captivates us all. By staying informed and engaged with the latest news and developments in space exploration, we can all play a part in shaping the future of humanity's journey to the stars.

*Stay updated on the latest space exploration news and developments by visiting NASA, SpaceX, ESA, and Space.com for the most up-to-date information.*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).